## このカーネルについて

2019/01開催[Kaggleタイタニックハンズオン（サポーターズ勉強会）](https://supporterzcolab.com/event/677/)用のkernelです。  
ハンズオンパートで使います

1. まず一緒に一回やってkernelの操作に慣れる
2. もくもくタイムで各自試行錯誤（コード片を組み合わせる）

## 基本操作

コードが書かれているブロックを **セル** と呼びます。  

1. セルをクリックして選択する（入力できる状態になる）
2. 選択したセルを実行する
    - 入力できる状態でShift+Enterキーを押す（こちらに慣れると簡単です）
    - 左側に表示される再生ボタン▶をクリック

## このカーネルでやること

1. 学習用データから乗客の生存／死亡を予測するモデルを作る
    1. 前処理
    2. モデル作成
    3. 性能確認
2. 1で作ったモデルでテスト用データについて予測する（その後、Kaggleに提出する）

## 1-A. 前処理

1. 欠損値
1. カテゴリ変数
1. 不要な列を削除

In [ ]:
# 前処理に必要なモジュールの読み込み
import numpy as np
import pandas as pd
# 可視化に必要なモジュールの読み込み
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

タイタニックコンペでは以下のフォルダ配置となる。これを踏まえて、相対パスでデータを指定する

- ┣ 現在のkernel
- ┗ input/
    - ┣ train.csv（学習用データ）
    - ┣ test.csv（テスト用データ）
    - ┗ gender_submission.csv（提出練習用データ）

In [ ]:
# 読み込んだデータはExcelの表のような形式で扱う（行と列がある）
# 学習用データの読み込み（生存か死亡か知っているデータ）
train_df = pd.read_csv('https://raw.githubusercontent.com/ftnext/spzcolab_titanic/master/input/train.csv')
# テスト用データの読み込み（生存か死亡か知らないデータ）
test_df = pd.read_csv('https://raw.githubusercontent.com/ftnext/spzcolab_titanic/master/input/test.csv')

`read_csv`  
解説: https://note.nkmk.me/python-pandas-read-csv-tsv/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [ ]:
# 学習用データのサイズを確認
# (行数, 列数) で表示される
train_df.shape

In [ ]:
# テスト用データのサイズを確認
# 学習用データに対して1列少ない
test_df.shape

`shape`  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shape.html

In [ ]:
# 学習用データの上から5行を表示
# 参考: train_df.head(7) # 上から7行表示
train_df.head()

In [ ]:
# テスト用データの上から5行を表示
# Survivedの列（生存か死亡かを表す）がないことが確認できる
test_df.head()

`head`  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html

In [ ]:
# 学習用データの情報を確認
train_df.info()

In [ ]:
# テスト用データの情報を確認
test_df.info()

`info`  
解説: https://note.nkmk.me/python-pandas-len-shape-size/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html

| infoによる情報 | 意味 |
| ----|---- | 
| int | 整数 |
| float | 浮動小数点数 |
| object | 文字列 |

| データの情報 | 列名 |　意味 | モデル作成に使う | 
| ----- | ----- | ----- | ----- |
| int | PassengerId | 乗客ID | |
| int | Pclass | チケットの等級 (1, 2, 3) | ◯ |
| int | SibSp | 同乗した兄弟姉妹/配偶者の人数 | ◯ |
| int | Parch | 同乗した両親/子供の人数 | ◯ |
| int | Survived | 0：死亡、1：生存 | 予測したい |
| float | Age | 年齢（推測があるため、浮動小数点数） | ◯ |
| float | Fare | 運賃 | ◯ |
| object | Name | 氏名 | |
| object | Sex | 性別 | ◯ |
| object | Ticket | チケット番号 | |
| object | Cabin | 船室番号　| |
| object | Embarked | 乗船した港の頭文字(S, Q, C) | ◯ |

**注**: モデル作成に使う列は、ハンズオン進行の都合により、こちらで決めました（本来はデータを分析してから決めます）  
ref: https://www.kaggle.com/c/titanic/data

In [ ]:
# 学習データについて欠けたデータがある列を確認（infoの情報に、891よりも少ない数の列があった）
train_df.isnull().sum()

In [ ]:
# テスト用データについて欠けたデータがある列を確認
test_df.isnull().sum()

`isnull().sum()`  
参考: https://note.nkmk.me/python-pandas-nan-judge-count/

欠けたデータがある列

| データ種別 | 列名 | 欠け具合 |
| ----- | ----- | ----- |
| 学習用・テスト用 | Age | 20%程度 |
| 学習用 | Embarked | 数件 |
| テスト用 | Fare | 数件 |
| 学習用・テスト用 | Cabin | 78%程度 |

- 欠けたデータ＝**欠損値**（missing value、欠測値とも呼ばれる）  
- 欠損の要因：データの収集過程で抜けてしまったなど
- 機械学習のツールは **一般に欠損値に対処できない**（そのためデータを前処理する）
- 削除すると貴重なデータが減るので、**埋める**

### 1-A 1.欠損値を埋める

- Age
- Fare
- Embarked

注: Cabinは欠損が多いため、使わないという選択をしました（そのため埋めません）

埋めるのに使うデータ

- Age: 学習用データの平均値（=年齢の総和/個数）
- Fare: 学習用データの中央値（運賃を小さい順に並べて真ん中に来る値）
- Embarked: 学習用データで一番多く登場する値（一番多くの人が乗っている港）

In [ ]:
# 学習用データの値を使って欠損を埋めるために使う値を表示
train_df[['Age', 'Fare', 'Embarked']].describe([.5], 'all')

- Age: 学習用データの平均値(mean) → 29.69... → 30歳で埋める
- Fare: 学習用データの中央値（50%） → 14.4542
- Embarked: 学習用データで一番多く登場する値（top） → S

`describe`  
参考: https://note.nkmk.me/python-pandas-describe/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html

In [ ]:
# 参考: Ageの分布（欠損値を除いて描画）
sns.distplot(train_df['Age'].dropna(), kde=False, bins=20)
plt.show()

In [ ]:
# 参考: Fareの分布
sns.distplot(train_df['Fare'], kde=False, bins=50)
plt.show()

In [ ]:
# Ageの欠損を平均値 30歳 で埋める
# **Note**: モクモクタイムで他の埋め方を試す際は、このセルを置き換えます
train_df['Age'] = train_df['Age'].fillna(30)
test_df['Age'] = test_df['Age'].fillna(30)

In [ ]:
# Embarkedの欠損を、一番多い乗船港 S で埋める
train_df['Embarked'] = train_df['Embarked'].fillna('S')

In [ ]:
# Fareの欠損を 中央値 14.4542 で埋める
test_df['Fare'] = test_df['Fare'].fillna(14.4542)

`fillna`  
参考: https://note.nkmk.me/python-pandas-nan-dropna-fillna/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

In [ ]:
# 学習用データの欠損値が埋まったことを確認
train_df.isnull().sum()

In [ ]:
# テスト用データの欠損値が埋まったことを確認
test_df.isnull().sum()

### 1-A 2.カテゴリ変数を数値化する

- Sex（male, female）
- Embarked(S, Q, C)

- カテゴリ変数とは、いくつかの文字列の値を取る変数
    - Sex: male, female
    - Embarked: S, Q, C　（乗船した港の頭文字）
- 文字列を整数に変換する
    - Sex: male=0, female=1として置き換え
    - Embarked: ダミー変数化（後述）

In [ ]:
# カテゴリを整数に置き換えるための辞書を用意
gender_map = {'female': 1, 'male': 0}
# 引数の辞書のキーに一致する要素が、辞書の値に置き換わる（femaleが1に置き換わり、maleが0に置き換わる）
# 注: Sexの取りうる値はfemaleかmale
train_df['Sex'] = train_df['Sex'].map(gender_map)
test_df['Sex'] = test_df['Sex'].map(gender_map)

`map`  
参考: https://note.nkmk.me/python-pandas-map-replace/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html

Embarkedのダミー変数化について

- S=1, Q=2, C=3と整数に置き換える
    - **本来なかった大小関係が想定されてしまう**
- (S, Q, C)という形式で整数に置き換える

| Embarkedの値 | 置き換えたあと |
| ----- | ----- |
| S | (1, 0, 0) |
| Q | (0, 1, 0) |
| C | (0, 0, 1) |

In [ ]:
# Embarked（S, Q, Cという3カテゴリ）をダミー変数にする
train_df = pd.get_dummies(train_df, columns=['Embarked'])
test_df = pd.get_dummies(test_df, columns=['Embarked'])

`get_dummies`  
参考: https://note.nkmk.me/python-pandas-get-dummies/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

### 1-A 3.不要な列を削除

In [ ]:
# 取り除く列のリスト
not_use_columns = ['Name', 'Ticket', 'Cabin']
# 学習用データから列を削除する（PassengerIdは後ほど取り除く）
train_df.drop(not_use_columns, axis=1, inplace=True)
# テスト用データから列を削除する
test_df.drop(not_use_columns, axis=1, inplace=True)

`drop`  
参考: https://note.nkmk.me/python-pandas-drop/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [ ]:
# 前処理した学習用データの確認
train_df.head()

In [ ]:
# 前処理したテスト用データの確認
test_df.head()

## 1-B. モデル作成

モデルは学習用データにアルゴリズムを適用して作成する。  
今回はアルゴリズムにロジスティック回帰を使う（後ほど変更も試せます）  

ドキュメント(ロジスティック関数): https://scikit-learn.org/stable/auto_examples/linear_model/plot_logistic.html

In [ ]:
# 慣例にのっとり、モデルが予測に使うデータをX, モデルが予測するデータをyとする
X = train_df.drop(['PassengerId', 'Survived'], axis=1)
y = train_df['Survived']

In [ ]:
# モデル作成・性能評価に使うモジュールの読み込み
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 

このあとモデルを作成するが、テスト用データについて生死を予測する前に、  
どの程度の性能のモデルなのか確認したい。  
→学習用データをランダムに2つに分ける（`train_test_split`）

- 学習用データのうち、例えば7割でモデルを作る
- 残りの3割でモデルの性能を評価する

参考情報

- ランダムに分けているが、`random_state`引数の値が同じなら、**何回実行しても同じ** ようにランダムに分かれる

In [ ]:
# 今回のハンズオンは7:3に分けて進める
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

`train_test_split`  
参考: https://docs.pyq.jp/python/machine_learning/tips/train_test_split.html  
ドキュメント: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

| 変数名 | 用途 |
| ----- | ----- |
| X_train | モデル作成用のデータ（乗客の属性） |
| y_train | モデル作成用のデータ（生死） |
| X_val | モデルの性能確認用のデータ（乗客の属性） |
| y_val | モデルの性能確認用のデータ（生死） |

In [ ]:
# モデル作成用のデータの数の確認
len(y_train)

In [ ]:
# モデル性能確認用のデータの数の確認
len(y_val)

In [ ]:
# ロジスティック回帰というアルゴリズムを使ったモデルを用意
model = LogisticRegression(random_state=1, solver='liblinear')
# モデル作成は以下の1行（ここまでの前処理に対してたった1行！）で完了する
model.fit(X_train, y_train)

## 1-C. 性能確認

タイタニックの場合、モデルの性能はaccuracyというスコアで評価される  
（**注**：性能を表すスコアは他にもある）

- 418名のうち生死を正しく予想できたものの割合
- 1に近いほど性能がよい

| ケース | 正解／誤り |
| ----- | ----- |
| 生存した乗客を生存と予測 | 正解 |
| 生存した乗客を死亡と予測 | 誤り |
| 死亡した乗客を生存と予測 | 誤り |
| 死亡した乗客を死亡と予測 | 正解 |

accuracy = (正解の総数) / (正解の総数 + 誤りの総数)

>Your score is the percentage of passengers you correctly predict. This is known simply as "accuracy”.

ref: https://www.kaggle.com/c/titanic#evaluation

In [ ]:
# モデル性能確認用データについて生死を予測
pred = model.predict(X_val)
# accuracyを算出して表示
accuracy_score(y_val, pred)

`accuracy_score`  
ドキュメント: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

初回はすぐ提出。  
別のモデルを作ったときに、以前に作ったモデルよりスコアが低ければ、提出しないという判断ができる  
（Kaggleに提出しなくても試行錯誤が進められる）

## 2. テスト用データの生死を予測

- テスト用データ（418件）について予測したあと、提出用データを作る
- 提出用データは以下の形式のCSVとするように指定されている（1が生存、0が死亡）

PassengerId | Survived
----- | -----
892 | 0
: | :
1309|1

- ref: https://www.kaggle.com/c/titanic#evaluation

In [ ]:
# テスト用データからPassengerId列を除く
X_test = test_df.drop(['PassengerId'], axis=1)
# テスト用データについて生死を予測
pred = model.predict(X_test)

In [ ]:
# colabでの実行の際は省略
# # 提出用データの形式に変換
# submission = pd.DataFrame({
#     'PassengerId': test_df['PassengerId'],
#     'Survived': pred
# })
# # 提出用データ作成
# submission.to_csv('submission.csv', index=False)

## 参考: 提出練習データを提出してみる

In [ ]:
# 以下のコードはお手元では実行不要です
# import pandas as pd
# gender_submission_df = pd.read_csv('../input/gender_submission.csv')
# gender_submission_df.to_csv('submission.csv', index=False)

## 精度を上げるために

精度が変わる方法（上がらない場合もあります）

- Ageの欠損の埋め方を見直す
- モデルのアルゴリズムを変える

### Ageの欠損の埋め方を見直す

- （案1） 中央値 28歳 で埋める
- （案2） Sexに応じて年齢を埋める
- （案3） Pclassに応じて年齢を埋める

KernelをEditする際、平均値で埋めた部分を以下のいずれかに置き換えてみてください。  
現状コメント扱いにしているため、コードとして実行するには、始めと終わりの`"""`を削除する必要があります

In [ ]:
# （案1） 中央値 28歳 で埋める
"""
train_df['Age'] = train_df['Age'].fillna(28)
test_df['Age'] = test_df['Age'].fillna(28)
"""

In [ ]:
# (案2) 仮説: 年齢の平均値は性別ごとに違うのでは？
# 性別ごとの年齢の平均値を確認
# train_df[['Sex', 'Age']].groupby('Sex').mean()

In [ ]:
# （案2）確認すると、男性の平均年齢 31歳、女性の平均年齢 28歳
"""
def age_by_sex(col):
    '''col: [age, sex]と想定'''
    age, sex = col
    if pd.isna(age): # Ageが欠損の場合の処理
        if sex == 'male':
            return 31
        elif sex == 'female':
            return 28
        else: # 整数に変更したsexが含まれる場合など
            print('Sexがmale/female以外の値をとっています')
            return -1
    else: # Ageが欠損していない場合の処理
        return age
# train_dfからAgeとSexの2列を取り出し、各行についてage_by_sex関数を適用
# age_by_sex関数の返り値でAge列の値を上書きする（欠損の場合は、値が埋められる）
train_df['Age'] = train_df[['Age', 'Sex']].apply(age_by_sex, axis=1)
test_df['Age'] = test_df[['Age', 'Sex']].apply(age_by_sex, axis=1)
"""

In [ ]:
# (案3) 仮説: 年齢の平均値はチケットの階級ごとに違うのでは？（年齢高い→お金持っている→いいチケット）
# チケットの等級ごとの年齢の平均値を確認
# train_df[['Pclass', 'Age']].groupby('Pclass').mean()

In [ ]:
# （案3） pclass==1 38歳、pclass==2 30歳、pclass==3 25歳
"""
def age_by_pclass(col):
    '''col: [age, pclass]と想定'''
    age, pclass = col
    if pd.isna(age): # Ageが欠損の場合の処理
        if pclass == 1:
            return 38
        elif pclass == 2:
            return 30
        else: # pclass == 3に相当する
            return 25
    else: # Ageが欠損していない場合の処理
        return age
train_df['Age'] = train_df[['Age', 'Pclass']].apply(age_by_pclass, axis=1)
test_df['Age'] = test_df[['Age', 'Pclass']].apply(age_by_pclass, axis=1)
"""

`groupby`  
参考: https://note.nkmk.me/python-pandas-groupby-statistics/  
ドキュメント: http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

`isna`  
ドキュメント: http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isna.html

`apply`  
参考: https://note.nkmk.me/python-pandas-map-applymap-apply/  
ドキュメント: http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

### モデルのアルゴリズムを変える

決定木を試す  

参考: https://scikit-learn.org/stable/modules/tree.html#classification  
ドキュメント: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [ ]:
"""
# 決定木というアルゴリズムを使ったモデルを用意
model = DecisionTreeClassifier(random_state=1)
# モデル作成は以下の1行（ここまでの前処理に対してたった1行！）で完了する
model.fit(X_train, y_train)
"""